In [12]:
#pip install --user --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [13]:
#pip install nest_asyncio

In [14]:
import socket
import sys
import requests
import json
import twint
from datetime import datetime, timedelta
from time import sleep


In [15]:
import nest_asyncio
nest_asyncio.apply()
# Tweets de las 2 últimas horas
current_date = datetime.now()
current_end_date = current_date - timedelta(minutes=240)
def twint_to_pandas(columns):
 return twint.output.panda.Tweets_df[columns]
# Configuración de twint
c= twint.Config()
c.Username = "el_pais"
c.Since = current_end_date.strftime("%Y-%m-%d %H:%M:%S")
c.Until = current_date.strftime("%Y-%m-%d %H:%M:%S")
c.Pandas = True
# Run
twint.run.Search(c)
df_pd = twint_to_pandas(["date", "username", 
 "tweet", "hashtags", 
"nlikes"])

1597913295075172352 2022-11-30 11:19:51 +0000 <el_pais> 📺 TV en DIRECTO | Marlaska: "Esto no es que el ministro de Interior se ponga detrás de las fuerzas y cuerpos de seguridad; se pone delante de ellas y defiende su actuación"  https://t.co/0rTRG60ECl  https://t.co/0NBzIG3edg
1597912919122931712 2022-11-30 11:18:22 +0000 <el_pais> 🔴 ÚLTIMA HORA | Marlaska insiste en el Congreso en que no hubo muertes en territorio nacional en la tragedia de Melilla  https://t.co/wD1EO2YnPc  https://t.co/5L1f4tosle
1597911839886581761 2022-11-30 11:14:05 +0000 <el_pais> 📺 TV en DIRECTO | Marlaska: "La Guardia Civil actuó de forma más que proporcionada en Melilla (...) No hubo ninguna pérdida de vida humana en territorio nacional. Y lo reitero"  https://t.co/0rTRG60ECl  https://t.co/TRcBCX861O
1597911334540218368 2022-11-30 11:12:04 +0000 <el_pais> Jiang, que ha muerto hoy a los 96 años, llegó al poder tras las protestas de Tiananmen con el encargo de buscar el equilibrio entre las reformas aperturista

**Procesar los tweets**

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pandasToSparkDF").getOrCreate()
df_pd = spark.createDataFrame(df_pd)
lines = df_pd.select("tweet")
print(lines)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


DataFrame[tweet: string]


**Procesar los tweets**

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
import re
def preprocessing(lines):
     words = lines.na.replace("", None)
     words = lines.na.drop()
     words = lines.withColumn("tweet", F.regexp_replace("tweet", r'http\S+', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", '@\w+', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", '#', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", 'RT', ''))
     words = lines.withColumn("tweet", F.regexp_replace("tweet", ':', ''))
     full_tweet = words.toJSON()
     return full_tweet 


**Función para enviar los tweets al socket**

In [9]:
def send_tweets_to_spark(data, tcp_connection):
     for row in data.collect():
         line = row + "\n"
         try:
             tcp_connection.send(line.encode("utf-8"))
         except:
             e = sys.exc_info()[0]
             print("Error connection: %s" % e)

**Crear el socket donde se almacena la información**

In [10]:
TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
data = preprocessing(lines)
send_tweets_to_spark(data, conn)

Waiting for TCP connection...


KeyboardInterrupt: 